# Session 3: A deeper dive into imaging data

## Aside: Introduction

In the previous session, we saw how we could retrieve and display images from the survey. We also saw how we could extract metadata to convert pixel coordinates to WCS coordinates, then retrieve a list of sources detected within an image and overlay them as points.

While many use cases will be satisfied with the analysis performed by the Data Release Processing pipeline, there may come times when one wants to tweak some of the default analysis. Having access to pixel-level data does, of course, mean one can use whatever bespoke packages and analysis one wishes. There are, however, benefits in utilising the LSST Software Stack to re-perform some analysis, not least in terms of ensuring consistency, which is critical should one wish to adjust an intermediate analysis step then feed the data back into the pipeline for subsequent steps.

In this notebook, we'll start by retrieving a calibrated image, take a look at some more of its associated metadata (specifically its pixel mask), before adjusting some of the source detection config parameters and the re-performing the source detection step using the re-configured parameters.

**Note:** Again, most of what's here is a re-writing of what's available in the DP0 Tutorials, notably Tutorial Notebook 05.

## As per usual, start with importing the required modules...

For now, don't worry about the LSST Pipeline Task modules, we'll look at those in more detail as we use them. 

In [1]:
# Butler module
from lsst.daf.butler import Butler

# Displaying modules and setting plot parameters
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay

# Table modules
import lsst.afw.table as afwTable

# Base modules:
import lsst.daf.base as dafBase

# LSST Pipeline tasks
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

## Instantiate a butler, and retrieve a calibrated exposure (calexp)

In [2]:
# For DP0.2, we want the following configuration and collection:
config = "dp02"
collection = '2.2i/runs/DP0.2'

butler = Butler(config=config, collections=collection)

dataId = {'visit': 512055, 'detector': 75}
calexp = butler.get('calexp', dataId=dataId)

## Aside: Interactive image viewing with Firefly

So far, we've only displayed images in-line within notebooks. While this is useful for quick-looks, the ability to interactively explore an image cannot be understated. And while you're unlikely to want to view the millions of images that the LSST will contain, viewing some for sanity checks is incredibly useful. For this reason, the RSP contains an in-built interactive image viewer - FireFly - which you can start-up from a notebook.   

## View the calexp using FireFly...

In [3]:
#afwDisplay.setDefaultBackend('firefly')
#afw_display = afwDisplay.Display(frame=1)
#afw_display.mtv(calexp)

## Aside: Pixel mask metadata

A calexp is exactly that: a calibrated exposure. In order for the calibration step to take place, sources must be detected in the exposure in order to be compared to known reference sources that are used for calibration. Because the LSST Data Processing pipeline is extremely good at "book-keeping" -- by which I mean keeping track of all steps and the products of those steps -- it keeps a record which pixels in an image are associated with a detected source. This record is in the form of a pixel mask, which is contained as metadata associated (i.e., within) the calexp object.

Since, however, we are going to re-perform the source detection step in this notebook, we first need to remove the existing detection pixel mask to ensure we start with a clean slate...

## Clear the existing 'DETECTED' pixel mask

But first, what number is associated with the 'DETECTED' mask?

In [4]:
calexp.mask.getMaskPlaneDict()

{'BAD': 0,
 'CR': 3,
 'CROSSTALK': 9,
 'DETECTED': 5,
 'DETECTED_NEGATIVE': 6,
 'EDGE': 4,
 'INTRP': 2,
 'NOT_DEBLENDED': 10,
 'NO_DATA': 8,
 'SAT': 1,
 'SUSPECT': 7,
 'UNMASKEDNAN': 11}

In [5]:
calexp.mask.clearMaskPlane(5)

## And re-display the image...

In [6]:
#afw_display.mtv(calexp)

## But, actually, we need to entirely remove the DETECTED mask plane...

In [7]:
calexp.mask.removeAndClearMaskPlane('DETECTED')

# Source detection

We're now going to perform our own source detection on the above calexp using a re-configured version of the pipeline's detection algorithm.

## Aside: Table schemas

Before we can perform source detection, the data processing software needs to first know what properties of the sources you wish it to measure and put into a table. This information is provided by a "Table Schema". As you'll see, we'll build-up the table schema as we go along, but first we need to create a minimal schema that contains the absolute minimum information that will be delivered for each source: unique ID number, position, and ID of any parent source (if deblended)...

## Create a minimal schema...

In [8]:
schema = afwTable.SourceTable.makeMinimalSchema()
#schema ## Check out what's in a minimal schema, if you like.

## Aside: Tasks, and their configurations

One can think of the Data Processing pipeline as a series of tasks that is performed on the incoming raw exposures to produce the ultimate outputs (e.g., calibrated images and source tables). Each one of these tasks has a set of configuration parameters (their `ConfigClass`). Most of the time these parameters are set at default values, but it is possible to access them, alter them, and re-run the task with the reconfigured parameters.

Below, we obtain the config object from the SourceDetectionTask, re-define the detection threshold, then re-upload the new config object.

## Grab the source detection config object from the task,...

In [9]:
config = SourceDetectionTask.ConfigClass()
#help(config) ## And see what config parameters you can alter, if you want.

## ...re-define the threshold parameter...

In [10]:
config.thresholdValue = 100 # The default is 5-sigma; here I'm redefining it as 100-stddev!
config.thresholdType = 'stdev'

## ...and instantiate a new source detection task with the new config and schema.

In [11]:
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

## Do the same for the source deblender task...

In [12]:
config = SourceDeblendTask.ConfigClass() # We're just using the default config, so technically this isn't needed.
sourceDeblendTask = SourceDeblendTask(schema=schema, config=config)
#schema ## Notice how the schema "grows" as we add more tasks.

## ...and the source measurement task.

In [14]:
config = SingleFrameMeasurementTask.ConfigClass() # Again, this isn't technically needed.
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config)

## And finally (for this part at least), make an empty table using the schema which will hold our results...

In [15]:
sourceTable = afwTable.SourceTable.make(schema)

## First, run the source detection task,...
Providing it with the sourceTable, which informs the task of the schema...

In [16]:
result = sourceDetectionTask.run(sourceTable, calexp)

## ...and check the outputs.

In [17]:
#help(result) ## The help function tells you to use the getDict() method to see what's in "results".
#result.getDict() ## Use .getDict() to see what you can extract from "results".
sources = result.sources

## Aside: Why do none of my detected sources have coordinates?

You'll notice from the table above that none of sources returned by the detection algorithm have coordinates. Why is this? Well, coordinates are a *measured* property, whereas so far we've performed the *detection* step. As such, those coordinate columns will be populated with values once we've *measured* the detected sources. In that case, how does the measuring task know where our detected sources are? The pixel coordinates of the detected sources are in there: try executing `results.positive` (for positive 100-stddev outliers) instead of `result.sources`.

## Run the deblend task

In [18]:
sourceDeblendTask.run(calexp, sources)

## Run the measurement task

In [19]:
sourceMeasurementTask.run(exposure=calexp, measCat=sources)

## "Deep" copy the source table to make sure it's contiguous in memory, and display it...

In [20]:
sources = sources.copy(deep=True)
sources.asAstropy()

id,coord_ra,coord_dec,parent,deblend_nChild,deblend_deblendedAsPsf,deblend_psfCenter_x,deblend_psfCenter_y,deblend_psf_instFlux,deblend_tooManyPeaks,deblend_parentTooBig,deblend_masked,deblend_failed,deblend_skipped,deblend_rampedTemplate,deblend_patchedTemplate,deblend_hasStrayFlux,deblend_peak_center_x,deblend_peak_center_y,deblend_peakId,deblend_nPeaks,deblend_parentNPeaks,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_SdssCentroid_x,slot_Centroid_x,base_SdssCentroid_y,slot_Centroid_y,base_SdssCentroid_xErr,slot_Centroid_xErr,base_SdssCentroid_yErr,slot_Centroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianFlux_flag_badCentroid,base_LocalBackground_flag_badCentroid,base_NaiveCentroid_flag_badInitialCentroid,base_PsfFlux_flag_badCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,slot_Centroid_flag,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_GaussianFlux_flag_badCentroid_edge,base_LocalBackground_flag_badCentroid_edge,base_NaiveCentroid_flag_badInitialCentroid_edge,base_PsfFlux_flag_badCentroid_edge,base_SdssShape_flag_badCentroid_edge,base_Variance_flag_badCentroid_edge,slot_Centroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_GaussianFlux_flag_badCentroid_noSecondDerivative,base_LocalBackground_flag_badCentroid_noSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,base_SdssShape_flag_badCentroid_noSecondDerivative,base_Variance_flag_badCentroid_noSecondDerivative,slot_Centroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_GaussianFlux_flag_badCentroid_almostNoSecondDerivative,base_LocalBackground_flag_badCentroid_almostNoSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,base_SdssShape_flag_badCentroid_almostNoSecondDerivative,base_Variance_flag_badCentroid_almostNoSecondDerivative,slot_Centroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_GaussianFlux_flag_badCentroid_notAtMaximum,base_LocalBackground_flag_badCentroid_notAtMaximum,base_NaiveCentroid_flag_badInitialCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,base_SdssShape_flag_badCentroid_notAtMaximum,base_Variance_flag_badCentroid_notAtMaximum,slot_Centroid_flag_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_GaussianFlux_flag_badCentroid_near_edge,base_LocalBackground_flag_badCentroid_near_edge,base_NaiveCentroid_flag_badInitialCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,base_SdssShape_flag_badCentroid_near_edge,base_Variance_flag_badCentroid_near_edge,slot_Centroid_flag_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_GaussianFlux_flag_badCentroid_resetToPeak,base_LocalBackground_flag_badCentroid_resetToPeak,base_NaiveCentroid_flag_badInitialCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,base_SdssShape_flag_badCentroid_resetToPeak,base_Variance_flag_badCentroid_resetToPeak,slot_Centroid_flag_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_GaussianFlux_flag_badCentroid_badError,base_LocalBackground_flag_badCentroid_badError,base_NaiveCentroid_flag_badInitialCentroid_badError,base_PsfFlux_flag_badCentroid_badError,base_SdssShape_flag_badCentroid_badError,base_Variance_flag_badCentroid_badError,slot_Centroid_flag_badError,base_Blendedness_old,base_Blendedness_raw,base_Blendedness_raw_child_instFlux,base_Blendedness_raw_parent_instFl

## Re-display the calibrated image and check the new DETECTED mask...

In [21]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)
afw_display.mtv(calexp)

## ...and plot on the detected sources as open red circles.

In [27]:
afw_display.erase()
with afw_display.Buffering():
    for record in sources:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='red')

## Write the source table to a fits file for local download.

In [32]:
# There is an in-built fits writer...
sources.writeFits('sources100stddev.fits')

# But typically, I find the astropy tables more useful as fits...
sources.asAstropy().write('sources100stddev_astroPy.fits')